In [1]:
import pandas as pd
import numpy as np
import json

In [2]:
OUTPUT_DIR = "./../output/datasets/"
DATA_DIR = "./../data/"

In [4]:
notes = pd.read_feather(DATA_DIR + "mimiciv_icd10.feather")

In [5]:
notes

,note_id,_id,text,icd10_proc,icd10_diag,target,num_words,num_targets,raw_text,icd10_diag_titles,icd10_proc_titles
0,10000084-DS-17,23052089,name unit no admission date discharge date dat...,[],"[E78.5, F02.80, G31.83, R29.6, R44.1, Z85.46]","[E78.5, F02.80, G31.83, R29.6, R44.1, Z85.46]",1305,6,\nName: ___ Unit No: __...,"[Hyperlipidemia, unspecified, Dementia in othe...",[]
1,10000117-DS-21,22927623,name unit no admission date discharge date dat...,[],"[F41.9, I34.1, K21.9, K31.819, K44.9, M81.0, R...","[F41.9, I34.1, K21.9, K31.819, K44.9, M81.0, R...",626,9,\nName: ___ Unit No: ___\n...,"[Anxiety disorder, unspecified, Nonrheumatic m...",[]
2,10000117-DS-22,27988844,name unit no admission date discharge date dat...,[0QS734Z],"[E78.00, F41.9, G43.909, I34.1, K21.9, M81.0, ...","[0QS734Z, E78.00, F41.9, G43.909, I34.1, K21.9...",887,13,\nName: ___ Unit No: ___\n...,"[Pure hypercholesterolemia, unspecified, Anxie...",[Reposition Left Upper Femur with Internal Fix...
3,10000980-DS-24,25911675,name unit no admission date discharge date dat...,[],"[D50.0, E11.8, I12.9, I25.10, I25.2, I50.23, I...","[D50.0, E11.8, I12.9, I25.10, I25.2, I50.23, I...",1722,22,\nName: ___ Unit No: ___\n \nAdmi...,[Iron deficiency anemia secondary to blood los...,[]
4,10000980-DS-25,29659838,name unit no admission date discharge date dat...,[],"[D63.1, E11.21, E78.5, I12.9, I25.10, I25.2, I...","[D63.1, E11.21, E78.5, I12.9, I25.10, I25.2, I...",1547,16,\nName: ___ Unit No: ___\n \nAdmi...,"[Anemia in chronic kidney disease, Type 2 diab...",[]
...,...,...,...,...,...,...,...,...,...,...,...
122273,19999784-DS-7,26194817,name unit no admission date discharge date dat...,"[07DR3ZX, 0CJS8ZZ]","[C85.99, D47.2, D64.9, D72.819, E44.0, F17.210...","[07DR3ZX, 0CJS8ZZ, C85.99, D47.2, D64.9, D72.8...",2152,12,\nName: ___ Unit No: __...,"[Non-Hodgkin lymphoma, unspecified, extranodal...","[Extraction of Iliac Bone Marrow, Percutaneous..."
122274,19999784-DS-8,24935234,name unit no admission date discharge date dat...,[],"[D47.2, F17.210, G95.89, J38.01, M54.16, R13.1...","[D47.2, F17.210, G95.89, J38.01, M54.16, R13.1...",1284,8,\nName: ___ Unit No: __...,"[Monoclonal gammopathy, Nicotine dependence, c...",[]
122275,19999784-DS-9,23664472,name unit no admission date discharge date dat...,"[00UT0JZ, 02HV33Z, 0HQ6XZZ, 3E03305]","[C83.39, D47.2, E43., G96.0, G97.82, T81.32XA,...","[00UT0JZ, 02HV33Z, 0HQ6XZZ, 3E03305, C83.39, D...",1436,14,\nName: ___ Unit No: __...,"[Diffuse large B-cell lymphoma, extranodal and...",[Supplement Spinal Meninges with Synthetic Sub...
122276,19999828-DS-6,29734428,name unit no admission date discharge date dat...,"[02HV33Z, 0HBHXZZ, 0HBJXZZ, 0HR7X74, 3E0436Z]","[B96.20, D68.51, E11.9, F12.90, F41.9, I10., I...","[02HV33Z, 0HBHXZZ, 0HBJXZZ, 0HR7X74, 3E0436Z, ...",1689,27,\nName: ___ Unit No: ___...,[Unspecified Escherichia coli [E. coli] as the...,[Insertion of Infusion Device into Superior Ve...


In [6]:
distinct_elements = set()
for titles in notes["icd10_diag"]:
    distinct_elements.update(titles)
distinct_count = len(distinct_elements)
print(distinct_count)

5803


In [15]:
distinct_elements

{'Drug induced subacute dyskinesia',
 'Asymptomatic menopausal state',
 'Choleperitonitis',
 'Unspecified ovarian cyst, unspecified side',
 'Fat necrosis of breast',
 'Postprocedural subglottic stenosis',
 'Unilateral femoral hernia, with obstruction, without gangrene, not specified as recurrent',
 'Diseases of lips',
 'Atypical atrial flutter',
 'Carpal tunnel syndrome, left upper limb',
 'Unspecified atherosclerosis of native arteries of extremities, other extremity',
 'Restlessness and agitation',
 'Endothelial corneal dystrophy',
 'Bathroom of single-family (private) house as the place of occurrence of the external cause',
 'Dependence on respirator [ventilator] status',
 'Hypovolemia',
 'Retained cholelithiasis following cholecystectomy',
 'Fracture of angle of left mandible, initial encounter for closed fracture',
 'Other fracture of first thoracic vertebra, initial encounter for closed fracture',
 'Absence epileptic syndrome, not intractable, without status epilepticus',
 'Poiso

In [6]:
diagnoses = pd.read_csv(DATA_DIR + "d_icd_diagnoses.csv.gz", compression="gzip")

In [18]:
cod2ents = {}
for index, row in diagnoses.iterrows():
    cod2ents[row["long_title"]] = set()

In [21]:
with open(OUTPUT_DIR + "pairs.jsonl", 'r') as f:
    for line in f:
        pairs = json.loads(line)["pairs"]
        for pair in pairs:
            cod2ents[pair["label"]].add(pair["term"])



In [22]:
empty_sets_count = sum(1 for v in cod2ents.values() if not v)
print(empty_sets_count)

5229


In [23]:
total_elements = sum(len(terms) for terms in cod2ents.values())
print(total_elements)

186143


In [16]:
cod2ents["Unspecified severe protein-calorie malnutrition"]

{'Abnormal weight loss',
 'Acute severe protein calorie malnutrition',
 'Albumin',
 'Alternating agents',
 'Anasarca',
 'Anemia of chronic disease',
 'Anorexia',
 'Anxiety',
 'Artificial feedings',
 'Artificial nutrition',
 'Artificial nutrition (TPN)',
 'Bolus tube feeds',
 'Cachectic',
 'Cachectic appearance',
 'Cachectic frail elderly female',
 'Cachexia',
 'Cachexia follow-up',
 'Cachexia treatment',
 'Cachexia/malnutrition',
 'Chronic malnutrition',
 'Chronic severe protein calorie malnutrition',
 'Cycled tube feeds',
 'Dialysis',
 'Diarrhea',
 'Diminished appetite',
 'Dobhoff placement',
 'Electrolyte abnormalities',
 'Elemental tube feeds',
 'Ensure',
 'Ensure Enlive',
 'Ensure Plus',
 'Ensure supplementation',
 'Ensure supplements',
 'Enteral feeding',
 'Enteral nutrition',
 'FOLFOX',
 'Failure to Thrive',
 'Failure to thrive',
 'Fat Wasting',
 'Fatty acid deficiency',
 'Ferritin',
 'Folate',
 'Foley catheter',
 'Folic acid',
 'Follow-up with nutrition',
 'Frappes',
 'Glucerna'